In [1]:
import pyodbc
import pandas as pd
import time
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import json

%matplotlib inline

table_prefix = 'test'
limit = 10000
forward_days = 30

with open ('../params.json') as f:
    params = json.load(f)
    
table_prefix = params['table_prefix']
diseases = params['diseases']
case_limit = params['case_limit']
control_limit = params['control_limit']
min_enrollment = params['enrollment']
user = params['user']
num_icds = 1000

chunk = True

creds_file = "/home/" + user + "/creds.txt" 
creds = lines = [line.rstrip('\n') for line in open(creds_file)]

connection_string = ("Driver={ODBC Driver 17 for SQL Server};" + 
                     "server=" + creds[0] + ";" +
                     "domain=" + creds[1] + ";" +  
                     "database=" + creds[2] + ";" +
                     "uid=" + creds[3]  + ";" +
                     "pwd=" + creds[4] + ";" +
                     "ssl=require;")

cn = pyodbc.connect(connection_string, autocommit=True)
cursor = cn.cursor()

### Calculate top X ICDs 

In [9]:
cursor.execute("IF OBJECT_ID('" + user + ".dbo." + "top_diagnoses', 'U') IS NOT NULL DROP TABLE " +
               user + ".dbo." + "top_diagnoses;")

disease_str = "'" + "', '".join(diseases.values()) + "'"
print(str(disease_str))

query = ("SELECT Object as ICD, COUNT(1) as Count " +
         "INTO " + user + ".dbo." + "top_diagnoses " + 
         "FROM " + user + ".dbo." + table_prefix + "_diagnosis " +
         "WHERE fromIndex > 0 AND fromIndex <= " + str(forward_days) + " "
         "GROUP BY Object ORDER BY COUNT(1) DESC;")
print(query)
cursor.execute(query)
query = ("SELECT ICD, count FROM " + user + ".dbo." + "top_diagnoses ORDER BY Count DESC;")
print(query)

df = pd.read_sql_query(query, cn)
print(df.shape)

if num_icds > 0:
    df = df[:num_icds]

print(df.head(200))

if num_icds > 0:
    limit = num_icds
else:
    limit = df.shape[0]
print(limit)

'332.0', '333.1'
SELECT Object as ICD, COUNT(1) as Count INTO bkb12.dbo.top_diagnoses FROM bkb12.dbo.test_diagnosis WHERE fromIndex > 0 AND fromIndex <= 30 GROUP BY Object ORDER BY COUNT(1) DESC;
SELECT ICD, count FROM bkb12.dbo.top_diagnoses ORDER BY Count DESC;
(5049, 2)
        ICD  count
0     332.0   6243
1     401.9   5060
2     781.2   3509
3     401.1   2809
4    250.00   2805
5    427.31   2568
6     272.4   1918
7     333.1   1908
8    780.79   1835
9     599.0   1582
10    781.0   1576
11   728.87   1575
12    724.2   1508
13    799.9   1386
14      496   1323
15    244.9   1312
16    729.5   1185
17   414.00   1172
18    719.7   1160
19   414.01   1146
20    285.9   1122
21    780.4   1112
22      486   1106
23    428.0   1080
24    780.2   1050
25    723.1   1018
26      311   1010
27   786.05    948
28   786.50    925
29    784.0    917
..      ...    ...
170  V72.31    192
171  726.10    189
172   703.0    189
173   429.3    189
174   174.9    188
175  V45.81    187
176 

In [16]:
query = ("SELECT MemberNum, Object as ICD, 1 as Present FROM " + user + ".dbo." +
         table_prefix + "_diagnosis " +  
         "WHERE fromIndex >= 0 AND fromIndex <" + str(forward_days) + " AND " + 
         "Object IN (SELECT TOP " + str(limit) + " ICD FROM " + user + ".dbo.top_diagnoses " + 
         "ORDER BY Count DESC)")
         
print(query)
df = pd.read_sql_query(query, cn)
print(df.head())
print(df.shape)

SELECT MemberNum, Object as ICD, 1 as Present FROM bkb12.dbo.test_diagnosis WHERE fromIndex >= 0 AND fromIndex <30 AND Object IN (SELECT TOP 1000 ICD FROM bkb12.dbo.top_diagnoses ORDER BY Count DESC)
        MemberNum     ICD  Present
0  14087222804825  296.32        1
1  12444855550925   V57.1        1
2   2770513014425   174.9        1
3   3557267619025   799.9        1
4   6524235323825  793.89        1
(231544, 3)


In [17]:
# check if icds of interest are in df, add if not
for key, val in diseases.items():
    if val in df['ICD'].unique():
        print(key, "in")
    else:
        print(key, "out")
        query = ("SELECT MemberNum, DiagnosisCode as ICD, 1 as Present FROM gmw3.dbo.DistinctDiagnosis " +
                 "WHERE DiagnosisCode = '" + val + "';") 
                 
        print(df.shape)
        col = pd.read_sql_query(query, cn)
        df = df.append(col)
        print(df.shape)

(u'Parkinsons', 'in')
(u'EssentialTremor', 'in')


In [18]:
print(df.dtypes)
print(df.head())
print(df.shape)
pivoted_df = df.pivot_table(index='MemberNum', columns='ICD', values='Present', aggfunc='mean')
pivoted_df.fillna(0, inplace=True)
pivoted_df = pivoted_df.astype(int)
print(pivoted_df.head())
print(pivoted_df.shape)

MemberNum     int64
ICD          object
Present       int64
dtype: object
        MemberNum     ICD  Present
0  14087222804825  296.32        1
1  12444855550925   V57.1        1
2   2770513014425   174.9        1
3   3557267619025   799.9        1
4   6524235323825  793.89        1
(231544, 3)
ICD         008.45  038.12  038.9  041.11  041.12  041.7  041.89  042  053.9  \
MemberNum                                                                      
779804825        0       0      0       0       0      0       0    0      0   
2585508525       0       0      0       0       0      0       0    0      0   
2959048125       0       0      0       0       0      0       0    0      0   
4183588525       0       0      0       0       0      0       0    0      0   
4397623725       0       0      0       0       0      0       0    0      0   

ICD         070.54   ...    V72.83  V72.84  V76.11  V76.12  V76.2  V76.41  \
MemberNum            ...                                          

In [19]:
print(pivoted_df.sum(axis=0))
print(pivoted_df.sum(axis=1))

ICD
008.45     53
038.12      9
038.9      91
041.11     13
041.12     18
041.7      13
041.89      6
042        26
053.9      45
070.54     19
078.10     50
079.99     36
088.81     43
110.1     610
110.4      49
112.0      35
112.9      14
135        25
153.9      42
162.9      32
173.3      49
173.31     33
173.6      20
174.4      21
174.8      23
174.9     126
183.0       9
185       268
188.9      48
189.0      23
         ... 
V58.73     18
V58.78     39
V58.81    125
V58.82     73
V58.83    189
V62.84     31
V66.7      17
V67.00     39
V67.09     45
V67.59     44
V70.0     809
V71.09      9
V71.4      38
V71.89     55
V72.0      63
V72.31    218
V72.5      50
V72.6      28
V72.60    172
V72.81    142
V72.83    141
V72.84    104
V76.11     29
V76.12    337
V76.2     105
V76.41     51
V76.44    185
V76.51    198
V77.91     49
V82.81     42
Length: 1000, dtype: int64
MemberNum
779804825              6
2585508525             3
2959048125             5
4183588525             9
43976

In [20]:
pivoted_df['label'] = 0

# get sum of disease columns (and exclude disease in multiple)
# if single_disease:
#     key_list = [val for key, val in diseases.items()]
#     pivoted_df = pivoted_df[pivoted_df[key_list].sum(axis=1) == 1]

# check to see how many patients remain
print(pivoted_df.shape)    

for key, val in diseases.items():
    print(val)
    print(pivoted_df[val].head())
    
    pivoted_df.loc[pivoted_df[val]>0, 'label'] = val
    pivoted_df.drop(val, axis=1, inplace=True)

print(pivoted_df['label'].value_counts())
pivoted_df = pivoted_df[pivoted_df['label'] != 0]
print(pivoted_df.shape)

(20000, 1001)
332.0
MemberNum
779804825     1
2585508525    1
2959048125    0
4183588525    1
4397623725    1
Name: 332.0, dtype: int64
333.1
MemberNum
779804825     0
2585508525    0
2959048125    1
4183588525    0
4397623725    0
Name: 333.1, dtype: int64
333.1    10000
332.0    10000
Name: label, dtype: int64
(20000, 999)


In [21]:
name = '' 
for key, val in diseases.items():
    name += key + '_'
print(name[:-1])

pivoted_df.to_csv('/home/bkb12/data/clustering/' + name[:-1] + str(forward_days) + '.csv.gz', compression='gzip')

Parkinsons_EssentialTremor
